# [PetFinder.my - Pawpularity Contest](https://www.kaggle.com/c/petfinder-pawpularity-score)
> Predict the popularity of shelter pet photos

![](https://storage.googleapis.com/kaggle-competitions/kaggle/25383/logos/header.png)

# Idea:
* Basic idea of this notebook is to use only **Image** Feature.
* Tabular data will be merged on later Notebooks. 
* **Wandb** is integrated hence we can use this notebook to track which experiemnt is peforming better and also do error analysis using **Grad-CAM** at the end.


# Notebooks:
* train: [[TF] PetFinder: Image [TPU][Train] 🐶](https://www.kaggle.com/awsaf49/tf-petfinder-image-tpu-train)
* infer: [[TF] PetFinder: Image [TPU][Infer] 🐶](https://www.kaggle.com/awsaf49/tf-petfinder-image-tpu-infer)


# Content:
* Install Libraries.
* Import Libraries.
* Libraries Version Check
* Configuration.
* DEVICE Configs.
* Meta Data.
* Train-Test Distrubution
* Data Augmentation.
* Data Pipeline.
* Model Config.
* Inference





# Install Libraries

In [1]:
!pip install -q /kaggle/input/efficientnet-keras-dataset/efficientnet_kaggle

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


# Import Libraries

In [2]:
import pandas as pd, numpy as np, random,os, shutil
import tensorflow as tf, re, math
import tensorflow.keras.backend as K
import efficientnet.tfkeras as efn
import sklearn
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import yaml

from glob import glob
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.metrics import roc_auc_score

2021-09-27 19:10:52.928405: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


# Version Check

In [3]:
print('np:', np.__version__)
print('pd:', pd.__version__)
print('sklearn:', sklearn.__version__)
print('tf:',tf.__version__)
print('tfa:', tfa.__version__)

np: 1.19.5
pd: 1.2.5
sklearn: 0.23.2
tf: 2.4.1
tfa: 0.12.1


# Configuration

In [4]:
class CFG:
    # DBUG OR not
    debug = False
    
    # DEVICE
    device = 'GPU'


    # IMAGE SIZE
    img_size = [512, 512]

    # BATCH SIZE AND EPOCHS
    batch_size  = 32

    # CFG.augmentATION
    augment   = True
    transform = False

    # TRANSFORMATION
    fill_mode = 'nearest'
    rot    = 10.0
    shr    = 5.0
    hzoom  = 30.0
    wzoom  = 30.0
    hshift = 30.0
    wshift = 30.0

    # FLIP
    hflip = True
    vflip = False

    # CLIP [0, 1]
    clip = False

    # Dropout
    drop_prob   = 0.75
    drop_cnt    = 10
    drop_size   = 0.05

    #bri, contrast
    sat  = [0.7, 1.3]
    cont = [0.8, 1.2]
    bri  =  0.15
    hue  = 0.05

    # TEST TIME CFG.augmentATION STEPS
    tta = 5
    
    tab_cols    = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
                   'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
    target_col  = ['Pawpularity']

# DEVICE Configs

In [5]:
if CFG.device == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        CFG.device = "GPU"

if CFG.device != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if CFG.device == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

Using default strategy for CPU and single GPU
Num GPUs Available:  1
REPLICAS: 1


2021-09-27 19:10:57.885253: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-27 19:10:57.889001: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-27 19:10:57.931379: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-27 19:10:57.932004: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-09-27 19:10:57.932054: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-27 19:10:57.957861: I tensorflow/stream_executor/platform/def

# Base Path for Dataset

In [6]:
BASE_PATH = '/kaggle/input/petfinder-pawpularity-score'

# Meta Data

In [7]:
# Train Data
df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
df['image_path'] = BASE_PATH + '/train/' + df.Id + '.jpg'
display(df.head(2))

# Test Data
test_df  = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_df['image_path'] = BASE_PATH + '/test/' + test_df.Id + '.jpg'

display(test_df.head(2))

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,image_path
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,/kaggle/input/petfinder-pawpularity-score/trai...
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,/kaggle/input/petfinder-pawpularity-score/trai...


,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,image_path
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1,/kaggle/input/petfinder-pawpularity-score/test...
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0,/kaggle/input/petfinder-pawpularity-score/test...


## Train-Test Ditribution

In [8]:
print('train_files:',df.shape[0])
print('test_files:',test_df.shape[0])

train_files: 9912
test_files: 8


# Data Augmentation
Used simple augmentations, some of them may hurt the model.
* RandomFlip (Left-Right)
* No Rotation
* RandomBrightness
* RndomContrast
* Shear
* Zoom
* Coarsee Dropout/Cutout

In [9]:
def get_mat(shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    #rotation = math.pi * rotation / 180.
    shear    = math.pi * shear    / 180.

    def get_3x3_mat(lst):
        return tf.reshape(tf.concat([lst],axis=0), [3,3])
    
    # ROTATION MATRIX
#     c1   = tf.math.cos(rotation)
#     s1   = tf.math.sin(rotation)
    one  = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    
#     rotation_matrix = get_3x3_mat([c1,   s1,   zero, 
#                                    -s1,  c1,   zero, 
#                                    zero, zero, one])    
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)    
    
    shear_matrix = get_3x3_mat([one,  s2,   zero, 
                               zero, c2,   zero, 
                                zero, zero, one])        
    # ZOOM MATRIX
    zoom_matrix = get_3x3_mat([one/height_zoom, zero,           zero, 
                               zero,            one/width_zoom, zero, 
                               zero,            zero,           one])    
    # SHIFT MATRIX
    shift_matrix = get_3x3_mat([one,  zero, height_shift, 
                                zero, one,  width_shift, 
                                zero, zero, one])
    

    return  K.dot(shear_matrix,K.dot(zoom_matrix, shift_matrix)) #K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))                  

def transform(image, DIM=CFG.img_size):#[rot,shr,h_zoom,w_zoom,h_shift,w_shift]):
    if DIM[0]!=DIM[1]:
        pad = (DIM[0]-DIM[1])//2
        image = tf.pad(image, [[0, 0], [pad, pad+1],[0, 0]])
        
    NEW_DIM = DIM[0]
    
    rot = CFG.rot * tf.random.normal([1], dtype='float32')
    shr = CFG.shr * tf.random.normal([1], dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / CFG.hzoom
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / CFG.wzoom
    h_shift = CFG.hshift * tf.random.normal([1], dtype='float32') 
    w_shift = CFG.wshift * tf.random.normal([1], dtype='float32') 
    
    transformation_matrix=tf.linalg.inv(get_mat(shr,h_zoom,w_zoom,h_shift,w_shift))
    
    flat_tensor=tfa.image.transform_ops.matrices_to_flat_transforms(transformation_matrix)
    
    image=tfa.image.transform(image,flat_tensor, fill_mode=CFG.fill_mode)
    
    rotation = math.pi * rot / 180.
    
    image=tfa.image.rotate(image,-rotation, fill_mode=CFG.fill_mode)
    
    if DIM[0]!=DIM[1]:
        image=tf.reshape(image, [NEW_DIM, NEW_DIM,3])
        image = image[:, pad:DIM[1]+pad,:]
    image = tf.reshape(image, [*DIM, 3])    
    return image

def dropout(image,DIM=CFG.img_size, PROBABILITY = 0.6, CT = 5, SZ = 0.1):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image with CT squares of side size SZ*DIM removed
    
    # DO DROPOUT WITH PROBABILITY DEFINED ABOVE
    P = tf.cast( tf.random.uniform([],0,1)<PROBABILITY, tf.int32)
    if (P==0)|(CT==0)|(SZ==0): 
        return image
    
    for k in range(CT):
        # CHOOSE RANDOM LOCATION
        x = tf.cast( tf.random.uniform([],0,DIM[1]),tf.int32)
        y = tf.cast( tf.random.uniform([],0,DIM[0]),tf.int32)
        # COMPUTE SQUARE 
        WIDTH = tf.cast( SZ*min(DIM),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM[0],y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM[1],x+WIDTH//2)
        # DROPOUT IMAGE
        one = image[ya:yb,0:xa,:]
        two = tf.zeros([yb-ya,xb-xa,3], dtype = image.dtype) 
        three = image[ya:yb,xb:DIM[1],:]
        middle = tf.concat([one,two,three],axis=1)
        image = tf.concat([image[0:ya,:,:],middle,image[yb:DIM[0],:,:]],axis=0)
        image = tf.reshape(image,[*DIM,3])

#     image = tf.reshape(image,[*DIM,3])
    return image

## Data Pipeline
* Reads the raw file and then decodes it to tf.Tensor
* Resizes the image in desired size
* Chages the datatype to **float32**
* Caches the Data for boosting up the speed.
* Uses Augmentations to reduce overfitting and make model more robust.
* Finally, splits the data into batches.


In [10]:
def build_decoder(with_labels=True, target_size=CFG.img_size, ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.image.resize(img, target_size)
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.reshape(img, [*target_size, 3])

        return img
    
    def decode_with_labels(path, label):
        return decode(path), tf.cast(label, tf.float32)
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True, dim=CFG.img_size):
    def augment(img, dim=dim):
        img = transform(img,DIM=dim) if CFG.transform else img
        img = tf.image.random_flip_left_right(img) if CFG.hflip else img
        img = tf.image.random_flip_up_down(img) if CFG.vflip else img
        img = tf.image.random_hue(img, CFG.hue)
        img = tf.image.random_saturation(img, CFG.sat[0], CFG.sat[1])
        img = tf.image.random_contrast(img, CFG.cont[0], CFG.cont[1])
        img = tf.image.random_brightness(img, CFG.bri)
        img = dropout(img, DIM=dim, PROBABILITY = CFG.drop_prob, CT = CFG.drop_cnt, SZ = CFG.drop_size)
        img = tf.clip_by_value(img, 0, 1)  if CFG.clip else img         
        img = tf.reshape(img, [*dim, 3])
        return img
    
    def augment_with_labels(img, label):    
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, batch_size=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir="", drop_remainder=False):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    ds = tf.data.Dataset.from_tensor_slices(slices)
    ds = ds.map(decode_fn, num_parallel_calls=AUTO)
    ds = ds.cache(cache_dir) if cache else ds
    ds = ds.repeat() if repeat else ds
    if shuffle: 
        ds = ds.shuffle(shuffle, seed=CFG.seed)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)
    ds = ds.map(augment_fn, num_parallel_calls=AUTO) if augment else ds
    ds = ds.batch(batch_size, drop_remainder=drop_remainder)
    ds = ds.prefetch(AUTO)
    return ds

# Model Configs

In [11]:
BASE_DIRS = [
    (512, '/kaggle/input/tf-petfinder-image-tpu-train'),
]

MODEL_CONFIGS = []
for dim, base_dir in  BASE_DIRS:
    paths = sorted(glob(os.path.join(base_dir, '*h5')))
    if len(paths)==0:
        print('no model found for :',base_dir)
    MODEL_CONFIGS.append([dim, paths])

# Inference

In [12]:
print('='*35)
print('### Inference')
print('='*35)
preds=[]
for dim, model_paths in tqdm(MODEL_CONFIGS):
    test_paths = test_df.image_path.tolist()
    if len(test_paths)<=8:
        CFG.batch_size = 1
    elif dim>=768:
        CFG.batch_size = REPLICAS * 24
    elif dim>=640:
        CFG.batch_size = REPLICAS * 32
    else:
        CFG.batch_size = REPLICAS * 64
    dtest = build_dataset(
        test_paths, 
        batch_size=CFG.batch_size, repeat=True, 
        shuffle=False, augment=True if CFG.tta>1 else False, cache=False,
        decode_fn=build_decoder(with_labels=False, target_size=[dim,dim]),
        augment_fn=build_augmenter(with_labels=False, dim=[dim, dim])
    )
    for model_path in model_paths:
        print(f'Model: {model_path}')
        with strategy.scope():
            print('Loading Model...')
            model = tf.keras.models.load_model(model_path, compile=False)
        print('Predicting...');
        pred = model.predict(dtest, steps = CFG.tta*len(test_paths)/CFG.batch_size, verbose=1)
        pred = pred[:CFG.tta*len(test_paths),:]
        pred = np.mean(pred.reshape(CFG.tta, len(test_paths), -1), axis=0)
        preds.append(pred)
        print()
preds = np.mean(preds, axis=0)

### Inference


  0%|          | 0/1 [00:00<?, ?it/s]

2021-09-27 19:10:59.008847: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-27 19:10:59.009092: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-27 19:10:59.009406: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-27 19:10:59.010375: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-0

Model: /kaggle/input/tf-petfinder-image-tpu-train/fold-0.h5
Loading Model...
Predicting...


2021-09-27 19:11:05.609607: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-27 19:11:05.612798: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000185000 Hz
2021-09-27 19:11:07.201710: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-09-27 19:11:07.993172: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-09-27 19:11:08.003709: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


40/40 [==============================] - 9s 25ms/step

Model: /kaggle/input/tf-petfinder-image-tpu-train/fold-1.h5
Loading Model...
Predicting...
40/40 [==============================] - 3s 23ms/step

Model: /kaggle/input/tf-petfinder-image-tpu-train/fold-2.h5
Loading Model...
Predicting...
40/40 [==============================] - 2s 23ms/step

Model: /kaggle/input/tf-petfinder-image-tpu-train/fold-3.h5
Loading Model...
Predicting...
40/40 [==============================] - 3s 26ms/step

Model: /kaggle/input/tf-petfinder-image-tpu-train/fold-4.h5
Loading Model...
Predicting...
40/40 [==============================] - 3s 23ms/step



# Submission

In [13]:
pred_df = pd.DataFrame({'Id':test_df.Id,
                        'Pawpularity':preds.reshape(-1)})
sub_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
del sub_df['Pawpularity']
sub_df = sub_df.merge(pred_df, on='Id', how='left')
sub_df.to_csv('submission.csv',index=False)
sub_df.head(2)

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,0.071107
1,43a2262d7738e3d420d453815151079e,0.075045
